In [1]:
import os
import tensorflow as tf
import model

In [4]:
%%time
tf.reset_default_graph()
sess = tf.InteractiveSession()
model.Model(3, 256, 64, 2, 0.0001)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
saver.restore(sess, os.getcwd() + "/model-rnn-vector-huber.ckpt")

INFO:tensorflow:Restoring parameters from /home/husein/space/text-dataset/sentiment/model-rnn-vector-huber.ckpt
CPU times: user 816 ms, sys: 40 ms, total: 856 ms
Wall time: 811 ms


In [5]:
%%time
tf.reset_default_graph()
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

g=load_graph('frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
y = g.get_tensor_by_name('import/logits:0')
sess = tf.InteractiveSession(graph=g)

CPU times: user 72 ms, sys: 8 ms, total: 80 ms
Wall time: 75.8 ms


## Freeze model loaded more faster than dynamic model

In [9]:
from queue import Queue
import threading
import requests

def run_parallel_in_threads(target, args_list):
    globalparas = []
    result = Queue()
    def task_wrapper(*args):
        result.put(target(*args))
    threads = [threading.Thread(target = task_wrapper, args = args) for args in args_list]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    while not result.empty():
        globalparas.append(result.get())
    globalparas = list(filter(None, globalparas))
    return globalparas

def get_time(text, type_api, i):
    response = str(requests.get("http://192.168.0.102:8033/%s?text=%s"%(type_api, text)).content)
    return [response, i]

# Stress test 20 requests concurrently on dynamic graph

In [27]:
CONCURRENT = 20
threads = []
for i in range(CONCURRENT):
    threads.append(('Freeze model loaded more faster than dynamic model', 'dynamic', i,))
outputs = run_parallel_in_threads(get_time, threads)
total = 0
for i in outputs:
    total += float(i[0][2:-1])
    print('thread %d, time taken %f s'%(i[1], float(i[0][2:-1])))
    
print('total time taken %f s, average time taken %f s'%(total, total / CONCURRENT))

thread 0, time taken 0.040708 s
thread 2, time taken 0.072802 s
thread 1, time taken 0.092469 s
thread 4, time taken 0.108417 s
thread 7, time taken 0.125185 s
thread 9, time taken 0.146564 s
thread 17, time taken 0.160176 s
thread 3, time taken 0.196733 s
thread 6, time taken 0.212126 s
thread 10, time taken 0.230030 s
thread 11, time taken 0.250885 s
thread 15, time taken 0.269423 s
thread 13, time taken 0.290279 s
thread 18, time taken 0.308505 s
thread 19, time taken 0.327376 s
thread 5, time taken 0.364115 s
thread 8, time taken 0.380689 s
thread 14, time taken 0.395581 s
thread 12, time taken 0.417496 s
thread 16, time taken 0.430993 s
total time taken 4.820551 s, average time taken 0.241028 s


# Stress test 20 requests concurrently on static graph

In [29]:
CONCURRENT = 20
threads = []
for i in range(CONCURRENT):
    threads.append(('Freeze model loaded more faster than dynamic model', 'static', i,))
outputs = run_parallel_in_threads(get_time, threads)
total = 0
for i in outputs:
    total += float(i[0][2:-1])
    print('thread %d, time taken %f s'%(i[1], float(i[0][2:-1])))
    
print('total time taken %f s, average time taken %f s'%(total, total / CONCURRENT))

thread 0, time taken 0.052546 s
thread 1, time taken 0.087430 s
thread 2, time taken 0.111106 s
thread 4, time taken 0.129980 s
thread 5, time taken 0.139806 s
thread 8, time taken 0.164908 s
thread 10, time taken 0.169885 s
thread 13, time taken 0.209557 s
thread 16, time taken 0.213291 s
thread 18, time taken 0.236292 s
thread 6, time taken 0.267163 s
thread 9, time taken 0.311357 s
thread 14, time taken 0.314128 s
thread 11, time taken 0.327267 s
thread 19, time taken 0.364782 s
thread 7, time taken 0.382049 s
thread 12, time taken 0.384515 s
thread 15, time taken 0.427014 s
thread 17, time taken 0.428580 s
thread 3, time taken 0.461565 s
total time taken 5.183223 s, average time taken 0.259161 s


# Run 5 experiments on stress test 20 requests concurrently on dynamic graph

In [34]:
total_experiments = 0
for _ in range(5):
    CONCURRENT = 20
    threads = []
    for i in range(CONCURRENT):
        threads.append(('Freeze model loaded more faster than dynamic model', 'dynamic', i,))
    outputs = run_parallel_in_threads(get_time, threads)
    total = 0
    for i in outputs:
        total += float(i[0][2:-1])
    total_experiments += total
    
print('time taken to run experiments %f s, average %f s'%(total_experiments, total_experiments / 5))

time taken to run experiments 27.863281 s, average 5.572656 s


# Run 5 experiments on stress test 20 requests concurrently on static graph

In [35]:
total_experiments = 0
for _ in range(5):
    CONCURRENT = 20
    threads = []
    for i in range(CONCURRENT):
        threads.append(('Freeze model loaded more faster than dynamic model', 'static', i,))
    outputs = run_parallel_in_threads(get_time, threads)
    total = 0
    for i in outputs:
        total += float(i[0][2:-1])
    total_experiments += total
    
print('time taken to run experiments %f s, average %f s'%(total_experiments, total_experiments / 5))

time taken to run experiments 26.089081 s, average 5.217816 s
